In [1]:
#####################################################################
def calibrate(im, XYZ, uv):
    # TBD
    C = np.zeros((4,3))
    n_points = XYZ.shape[0]
    
    # generate scaling factor
    w_0 = 1
    w = np.ones((n_points,w_0))
    
    # transform to homogeneous system 
    X_raw = XYZ
    x_raw = np.hstack([uv,w])
    
    # normalization matrix
    # x normalize
    T_norm = np.array([[im.shape[0]+im.shape[1],0,im.shape[0]/2],[0,im.shape[0]+im.shape[1],im.shape[1]/2],[0,0,1]])
#     print(T_norm)
    T_norm = np.linalg.inv(T_norm)
    
    x = []
    for x_single in x_raw:
        tmp = T_norm@x_single
        x.append(tmp)
    x = np.array(x)
    
    # X normalize
    X_centralized_sum = []
    X_mean = np.sum(X_raw,axis=0)
    for i in range(n_points):        
        X_centralized = np.array([X_raw[i]-X_mean]).T @ np.array([X_raw[i]-X_mean])
        X_centralized_sum.append(X_centralized)
    X_centralized_sum = np.sum(X_centralized_sum,axis=0)
#     print(X_centralized_sum)
    eigenvalues_X, eigen_vectors_X = np.linalg.eig(X_centralized_sum)
#     print(eigenvalues_X)
#     print(eigen_vectors_X)
    V = eigen_vectors_X
    V_1 = np.linalg.inv(eigen_vectors_X)
    diag = np.diag([np.power(m,-1) for m in eigenvalues_X])
    component = V @ diag @ V_1
    S_norm = np.hstack([component,(-component@X_mean).reshape(3,1)])
    zeros_one =np.array([0,0,0,1]).reshape(1,4)
    S_norm = np.vstack([S_norm,zeros_one])
    print(S_norm.shape)
    
    X = []
    X_raw = np.hstack([X_raw,w])
    for X_single in X_raw:
        tmp = S_norm @ X_single
        X.append(tmp)
    X = np.array(X)

    
    # Factorizing unknowns w/ 2 equations
    A = []
    for i in range(n_points):
        zero_vector = np.zeros((4,1))
        fst_row = np.concatenate([zero_vector.T,-w_0*X[i].reshape(1,4),x[i][1]*X[i].reshape(1,4)],axis=1)
        snd_row = np.concatenate([w_0*X[i].reshape(1,4),zero_vector.T,-x[i][0]*X[i].reshape(1,4)],axis=1)
        A.append(fst_row)
        if i != n_points-1:
            A.append(snd_row)
    A = np.array(A)
    A = np.squeeze(A,axis=1)
#     print(A.shape)
    U,SIGMA,VH = np.linalg.svd(A)
#     print(U.shape)
#     print(SIGMA.shape)
#     print(VH.shape)
    p = VH.T[:,-1]
#     print(p)
    print("orthogonal check: ",np.sqrt(sum([item**2 for item in p])))
    P = p.reshape((3,4))
    print(P)
#     P = np.linalg.inv(T_norm) @ P @ S_norm
#     print(P)


#     U_P,SIGMA_P,VH_P = np.linalg.svd(P)
#     p_1 = P[:,0]
#     p_2 = P[:,1]
#     p_3 = P[:,2]
#     p_4 = P[:,3]
    
#     p_234 = P[:,1:]
#     p_134 = P[:,[0,2,3]]
#     p_124 = P[:,[0,1,3]]
#     p_123 = P[:,:3]
# #     print(p_123)
# #     print(np.linalg.det(p_123))
#     M = p_123
#     R,K = np.linalg.qr(M)
#     print("K\n ",K)
#     print("R\n ",R)
#     C = np.array([np.linalg.det(p_234),-np.linalg.det(p_134),np.linalg.det(p_124),-np.linalg.det(p_123)])
#     print("C\n ",C)
    
    
    K, R, t = vgg_KR_from_P(P)
    print("K\n ",K)
    print("R\n ",R)
    print("t\n",t)
    return C

In [ ]:
#     P = np.linalg.inv(T_norm) @ P @ S_norm
#     print(P)


#     U_P,SIGMA_P,VH_P = np.linalg.svd(P)
#     p_1 = P[:,0]
#     p_2 = P[:,1]
#     p_3 = P[:,2]
#     p_4 = P[:,3]
    
#     p_234 = P[:,1:]
#     p_134 = P[:,[0,2,3]]
#     p_124 = P[:,[0,1,3]]
#     p_123 = P[:,:3]
# #     print(p_123)
# #     print(np.linalg.det(p_123))
#     M = p_123
#     R,K = np.linalg.qr(M)
#     print("K\n ",K)
#     print("R\n ",R)
#     C = np.array([np.linalg.det(p_234),-np.linalg.det(p_134),np.linalg.det(p_124),-np.linalg.det(p_123)])
#     print("C\n ",C)
    

In [ ]:
f_x_resized = K_resized[0,0]
theta_resized = np.arctan(-f_x_resized/K_resized[0,1])
f_y_resized = K_resized[1,1] * np.sin(theta_resized)
overall_focal_length_resized = np.sqrt(f_x_resized ** 2 + f_y_resized ** 2)
# print(theta)
print(f_x_resized)
print(f_y_resized)
print(overall_focal_length_resized)

beta_resized = np.arctan(-R_resized[2,0]/np.sqrt(R_resized[2,1]**2 + R_resized[2,2]**2))
print(f"beta: {beta_resized} rad")
beta_angle_resized = beta_resized * 180 /np.pi
print(f"beta: {beta_angle_resized} degree")

In [ ]:
def calibrate(im, XYZ, uv):
    # TBD
    C = np.zeros((4,3))
    n_points = XYZ.shape[0]
    
    # generate scaling factor
    w_0 = 1
    
    # transform to homogeneous system 
    X = homogeneous_transform(XYZ,w_0)
    x = homogeneous_transform(uv,w_0)
#     print(X)
#     print(x)
    
    # Factorizing unknowns w/ 2 equations
    A = []
    for i in range(n_points):
        zero_vector = np.zeros((4,1))
        fst_row = np.concatenate([zero_vector.T,-w_0*X[i].reshape(1,4),x[i][1]*X[i].reshape(1,4)],axis=1)
        snd_row = np.concatenate([w_0*X[i].reshape(1,4),zero_vector.T,-x[i][0]*X[i].reshape(1,4)],axis=1)
        A.append(fst_row)
        A.append(snd_row)
#         if i != n_points-1:
#             A.append(snd_row)
    A = np.array(A)
    A = np.squeeze(A,axis=1)
#     print(A.shape)
    U,SIGMA,VH = np.linalg.svd(A)

    p = VH.T[:,-1]
    p = p/p[-1]
#     print("normalization check: ",np.sqrt(sum([item**2 for item in p]))) # we won't go for this since we ignore normalization procedure
    P = p.reshape((3,4))
    C = P
#     print(P)
    return C